In [1]:
# Importa la las librerias para llevar a cabo la autentificacion
# con Google Earth Engine.
import ee
# ee.Authorize()
ee.Initialize()

In [2]:
# Importacion de la libreria Folium (Integración de Leaflet con Python).
import folium
from folium import plugins
print(folium.__version__)

0.12.1


In [3]:
# Importacion de los assets correspondientes a el area de interes
Ciudad_de_Mexico = ee.FeatureCollection("users/juanalexis0407/ZM_CDMX")

In [4]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')	
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).clip(Ciudad_de_Mexico)

## Landsat Collection

In [5]:
col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
  .map(maskL8sr) \
  .filterDate('2020-01-01','2020-12-31') \
  .filterBounds(Ciudad_de_Mexico);

In [6]:
# Image reduction
image = col.median();

## NDVI

In [7]:
ndvi = image.normalizedDifference(['B5','B4']).rename('NDVI');

## Fractional vegetation

In [8]:
# Reduce the region. The region parameter is the Feature geometry.
minValue = ndvi.reduceRegion(**{
  'reducer': ee.Reducer.min(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(minValue.getInfo())

# Reduce the region. The region parameter is the Feature geometry.
maxValue = ndvi.reduceRegion(**{
  'reducer': ee.Reducer.max(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(maxValue.getInfo())

min_va = ee.Image(-0.709741550695825)
max_va = ee.Image(1)

{'NDVI': -0.709741550695825}
{'NDVI': 1}


In [9]:
fv =(ndvi.subtract(min_va).divide(max_va.subtract(min_va))).pow(ee.Number(2)).rename('FV');

## Calculo de Emisividad

### Thermal

In [10]:
thermal= image.select('B10').multiply(0.1)

In [11]:
# Reduce the region. The region parameter is the Feature geometry.
min_thermal = thermal.reduceRegion(**{
  'reducer': ee.Reducer.min(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(min_thermal.getInfo())

# Reduce the region. The region parameter is the Feature geometry.
max_thermal = thermal.reduceRegion(**{
  'reducer': ee.Reducer.max(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(max_thermal.getInfo())

{'B10': 272.45}
{'B10': 313.40000000000003}


### Emisividad

In [12]:
# Crea las constantes 
a = ee.Number(0.004);
b = ee.Number(0.986);

In [13]:
EM = fv.multiply(a).add(b).rename('EMM')

In [14]:
# Reduce the region. The region parameter is the Feature geometry.
min_EM = EM.reduceRegion(**{
  'reducer': ee.Reducer.min(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(min_EM.getInfo())

# Reduce the region. The region parameter is the Feature geometry.
max_EM = EM.reduceRegion(**{
  'reducer': ee.Reducer.max(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(max_EM.getInfo())

{'EMM': 0.986}
{'EMM': 0.99}


### LST

In [15]:
LST = thermal.expression(
'(Tb/(1 + (0.00115* (Tb / 1.438))*log(Ep)))-273.15', {
 'Tb': thermal.select('B10'),
'Ep': EM.select('EMM')
}).rename('LST');

In [16]:
# Reduce the region. The region parameter is the Feature geometry.
min_LST = LST.reduceRegion(**{
  'reducer': ee.Reducer.min(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(min_LST.getInfo())

# Reduce the region. The region parameter is the Feature geometry.
max_LST = LST.reduceRegion(**{
  'reducer': ee.Reducer.max(),
  'geometry': Ciudad_de_Mexico.geometry(),
  'scale': 30,
  'crs': 'EPSG:4326',
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print(max_LST.getInfo())

{'LST': 0.08919025379753975}
{'LST': 41.29975039804117}


## Folium map

In [17]:
# RGB
mapid_1 = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}) # Mosaico RGB
center = [19.4338, -99.13261]
map = folium.Map(location=center, zoom_start=13)

folium.TileLayer(
    tiles=mapid_1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Color Verdadero',
  ).add_to(map)

# NDVI
mapid_2 = ndvi.getMapId({'palette': ['red', 'white', 'green'], 'min': -1, 'max': 1}) # Mosaico NDVI
folium.TileLayer(
    tiles=mapid_2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  ).add_to(map)

# fv
mapid_3 = fv.getMapId({'min': -1, 'max': 1}) # Mosaico fv 
folium.TileLayer(
    tiles=mapid_3['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='fv',
  ).add_to(map)

# thermal
mapid_4 = thermal.getMapId({'min': 272, 'max': 314, 'palette': ['blue','white', 'green']}) # banda thermal 
folium.TileLayer(
    tiles=mapid_4['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='thermal',
  ).add_to(map)

# EMM
mapid_5 = EM.getMapId({'min': 0.98, 'max': 0.99}) # Mosaico Emisividad 
folium.TileLayer(
    tiles=mapid_5['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Emisividad',
  ).add_to(map)

# LST
mapid_6 = LST.getMapId({'min': 0.09, 'max':42, 'palette': [
'040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
'0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
'3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
'ff0000', 'de0101', 'c21301', 'a71001', '911003'
 ], 'opacity': 0.5}) # Mosaico LST 
folium.TileLayer(
    tiles=mapid_6['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='LST',
  ).add_to(map)

map.add_child(folium.LayerControl())

#mouse position
fmtr = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
plugins.MousePosition(position='bottomright', separator=' | ', prefix="Mouse:",lat_formatter=fmtr, lng_formatter=fmtr).add_to(map)

#Add the draw 
plugins.Draw(export=True, filename='data.geojson', position='topleft', draw_options=None, edit_options=None).add_to(map)  

#Add measure tool 
plugins.MeasureControl(position='topright', primary_length_unit='meters', secondary_length_unit='miles', primary_area_unit='sqmeters', secondary_area_unit='acres').add_to(map)

map

In [18]:
# Guardar mapa en un archivo.
## map.save('Ciudad_de_Mexico_LST.html')